# Install Dependencies

In [ ]:
%pip install "google-cloud-aiplatform[agent_engines,adk]==1.88 " -q
%pip install -U google-adk==0.2.0  -q

print("Installation complete.")


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Installation complete.


In [ ]:
!pip list

Package                            Version
---------------------------------- --------------
annotated-types                    0.7.0
anyio                              4.9.0
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
arrow                              1.3.0
asttokens                          3.0.0
async-lru                          2.0.5
attrs                              25.3.0
Authlib                            1.6.0
babel                              2.17.0
beautifulsoup4                     4.13.4
bleach                             6.2.0
cachetools                         5.5.2
certifi                            2025.4.26
cffi                               1.17.1
charset-normalizer                 3.4.2
click                              8.2.1
cloudpickle                        3.1.1
comm                               0.2.2
cryptography                       45.0.3
debugpy                            1.8.14
decorator                         

In [3]:
import os
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner


print("Libraries imported.")

Libraries imported.


In [20]:
GOOGLE_CLOUD_PROJECT = "development-459201" # @param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"


DEFAULT_MODEL = "gemini-2.0-flash"

# Create an Agent

In [21]:
root_agent = Agent(
    model=DEFAULT_MODEL,
    name="hello_world_agent",
    description="An agent that says 'hello world'",
    instruction="""You always say 'hello world' to the user, and nothing else.
    Output 'hello world' in a random language.
    Put the language in brackets.

    Example Output 1:
    hello world (English)
    """
)


In [7]:
session_service = InMemorySessionService()


APP_NAME = "hello_world_app"
USER_ID = "user_1"
SESSION_ID = "session_001" 


session = session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")


runner = Runner(
    agent=root_agent, 
    app_name=APP_NAME,  
    session_service=session_service 
)


Session created: App='hello_world_app', User='user_1', Session='session_001'


In [8]:
from google.genai import types

async def call_agent_async(query: str, runner, user_id, session_id):

  print(f"\n>>> User: {query}")

  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not generate a final response." 

  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      if event.is_final_response():
          if event.content and event.content.parts:
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: 
             final_response_text = f"Error: {event.error_message or 'No message'}"

          break 

  print(f"<<< Agent: {final_response_text}")

In [10]:
async def run_conversation():
    await call_agent_async("Hello",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)


await run_conversation()



>>> User: Hello


/home/vscode/.pyenv/versions/3.11.12/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


<<< Agent: こんにちは世界 (Japanese)



# Deploy to Agent Engine

In [11]:
import time
import vertexai
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp
from google.adk.agents import Agent
from typing import Generator, Iterable

STAGING_BUCKET = "gs://adk-mini-tap" #@param {type:"string"}

vertexai.init(
    staging_bucket=STAGING_BUCKET,
)

In [12]:
# VertexAI SDK の AdkApp クラスでエージェントをラップする
app = AdkApp(agent=root_agent)

/home/vscode/.pyenv/versions/3.11.12/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [13]:
remote_app = agent_engines.create(
    app,
    requirements=[
        "google-cloud-aiplatform[agent_engines,adk]==1.88",
        "google-adk==0.2.0"
    ],
    display_name="Hello World Agent",
    description="Agent Engine",
)

Identified the following requirements: {'google-cloud-aiplatform': '1.88.0', 'cloudpickle': '3.1.1'}
The following requirements are missing: {'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]==1.88', 'google-adk==0.2.0', 'cloudpickle==3.1.1']


/home/vscode/.pyenv/versions/3.11.12/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Using bucket adk-mini-tap
Wrote to gs://adk-mini-tap/agent_engine/agent_engine.pkl
Writing to gs://adk-mini-tap/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://adk-mini-tap/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/1017461389635/locations/us-central1/reasoningEngines/5258137282908520448/operations/3905041268329152512
View progress and logs at https://console.cloud.google.com/logs/query?project=development-459201
AgentEngine created. Resource name: projects/1017461389635/locations/us-central1/reasoningEngines/5258137282908520448
To use this AgentEngine in another session:
agent_engine = vertexai.agent_engines.get('projects/1017461389635/locations/us-central1/reasoningEngines/5258137282908520448')


In [16]:
print(f"Type of remote_app object: {type(remote_app)}")
print(f"Remote app has 'query' attribute: {hasattr(remote_app, 'query')}")
print(f"Remote app has 'async_query' attribute: {hasattr(remote_app, 'async_query')}")
print(f"Remote app has 'stream_query' attribute: {hasattr(remote_app, 'stream_query')}")
print(f"Remote app has 'async_stream_query' attribute: {hasattr(remote_app, 'async_stream_query')}")


Type of remote_app object: <class 'vertexai.agent_engines._agent_engines.AgentEngine'>
Remote app has 'query' attribute: False
Remote app has 'async_query' attribute: False
Remote app has 'stream_query' attribute: True
Remote app has 'async_stream_query' attribute: False


In [15]:
remote_app.resource_name

'projects/1017461389635/locations/us-central1/reasoningEngines/5258137282908520448'

In [17]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

{'events': [],
 'last_update_time': 1748530595.921835,
 'state': {},
 'id': '5759660270465056768',
 'app_name': '5258137282908520448',
 'user_id': 'u_456'}

In [18]:
remote_app.get_session(user_id="u_456", session_id=remote_session["id"])

{'events': [],
 'last_update_time': 1748530595.921835,
 'state': {},
 'id': '5759660270465056768',
 'app_name': '5258137282908520448',
 'user_id': 'u_456'}

In [19]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="Hello World",
):
    print(event)

ValueError: Can only parse array of JSON objects, instead got {"

In [ ]:
for remote_agent in agent_engines.list():
  print(remote_agent.resource_name)

In [20]:
# 削除
for remote_agent in agent_engines.list():
  remote_agent.delete(force=True)

Falling back to grpc since no async rest credentials were detected.


Delete Agent Engine backing LRO: projects/1017461389635/locations/us-central1/operations/4061259880403566592


INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/1017461389635/locations/us-central1/operations/4061259880403566592


Agent Engine deleted. Resource name: projects/1017461389635/locations/us-central1/reasoningEngines/103767509382987776


INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/1017461389635/locations/us-central1/reasoningEngines/103767509382987776
Falling back to grpc since no async rest credentials were detected.


Delete Agent Engine backing LRO: projects/1017461389635/locations/us-central1/operations/6103642311416086528


INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/1017461389635/locations/us-central1/operations/6103642311416086528


Agent Engine deleted. Resource name: projects/1017461389635/locations/us-central1/reasoningEngines/316562591776243712


INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/1017461389635/locations/us-central1/reasoningEngines/316562591776243712
